In [75]:
pip install openpyxl


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [76]:
pip install requests


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Data Collection – ENTSO-E API
This notebook uses the ENTSO-E Transparency Platform API to pull hourly day-ahead power prices for the UK and France.
We will:
- Use the official API to fetch historical data
- Parse the XML response into a clean DataFrame
- Save the cleaned dataset to data/processed/cleaned_UK_FR_prices.csv



Note:
- ENTSO-E returns data in XML format, not JSON or CSV
- Use requests + xml.etree.ElementTree to parse it
- Need API key set as a variable
- Each API call can return a max of 1 month of data - so we loop monthly

In [77]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [78]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import os, sys
import urllib3
import ssl
import numpy as np
sys.path.append(os.path.abspath("/Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main"))
from config import raw_data_dir, processed_data_dir, API_KEY, FR_DOMAIN, START_DATE, END_DATE, FX_GBP_EUR
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

In [79]:
#Check
print(raw_data_dir) 
print(processed_data_dir) 
print(API_KEY, FR_DOMAIN) 
print(START_DATE)
print(END_DATE)
print(FX_GBP_EUR)

/Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main/data/raw
/Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main/data/processed
3faef1ee-b130-4678-9759-4ac9c0af0941 10YFR-RTE------C
2025-01-01 00:00:00
2025-08-01 23:00:00
1.17


In [80]:
# Excel file name - Uk day ahead prices
uk_xlsx = os.path.join(raw_data_dir, "uk_day_ahead_prices.xlsx")


In [81]:
def fmt(dt: datetime) -> str:

    return dt.strftime("%Y%m%d%H%M")

def next_month(dt: datetime) -> datetime:

    y, m = dt.year, dt.month

    return datetime(y + (m == 12), 1 if m == 12 else m + 1, 1)

def month_windows(start: datetime, end: datetime):

    """

    Yield (m_start, m_end_exclusive) covering [start, end], with m_end_exclusive set to

    00:00 of the first day of the next month, per ENTSO-E best practice.

    We’ll clip the last window to end+1hour to be safe, then filter later.

    """

    cur = datetime(start.year, start.month, 1)

    while cur <= end:

        nxt = next_month(cur)

        yield max(cur, start), min(nxt, next_month(end.replace(day=1)))

        cur = nxt

def parse_resolution_to_timedelta(res_text: str) -> timedelta:

    # Expected "PT60M" for hourly. You can extend as needed.

    # Minimal parser: supports PT{n}M and PT{n}H

    if not res_text or not res_text.startswith("PT"):

        return timedelta(hours=1)

    body = res_text[2:]

    if body.endswith("M"):

        minutes = int(body[:-1])

        return timedelta(minutes=minutes)

    if body.endswith("H"):

        hours = int(body[:-1])

        return timedelta(hours=hours)

    # default to 1 hour if unknown

    return timedelta(hours=1)

def fetch_entsoe_fr_period(start_exclusive: datetime, end_exclusive: datetime) -> pd.DataFrame:

    """

    Request FR day-ahead prices for [start_exclusive, end_exclusive) per ENTSO-E convention.

    Parse using each Period's timeInterval start rather than assuming month start.

    """

    params = {

        "securityToken": API_KEY,

        "documentType": "A44",                 # Day-ahead prices

        "in_Domain": FR_DOMAIN,

        "out_Domain": FR_DOMAIN,

        "periodStart": fmt(start_exclusive),

        "periodEnd": fmt(end_exclusive),     

    }

    r = requests.get("https://web-api.tp.entsoe.eu/api", params=params, timeout=60, verify=False)

    r.raise_for_status()

    root = ET.fromstring(r.content)

    rows = []

    # For each TimeSeries → for each Period → get timeInterval start and resolution
    for ts in root.findall(".//{*}TimeSeries"):

        for period in ts.findall(".//{*}Period"):

            ti = period.find("{*}timeInterval")

            p_start_text = ti.find("{*}start").text


            res_text = period.find("{*}resolution").text if period.find("{*}resolution") is not None else "PT60M"

            p_start = datetime.fromisoformat(p_start_text.replace("Z",""))

            step = parse_resolution_to_timedelta(res_text)

            for pt in period.findall("{*}Point"):

                pos = int(pt.find("{*}position").text)

                price = float(pt.find("{*}price.amount").text)

                ts_dt = p_start + (pos - 1) * step

                rows.append({"datetime": ts_dt, "FR_price": price})

    df = pd.DataFrame(rows)

    if df.empty:

        return df

    # Normalize: floor to hour, sort, dedupe

    df["datetime"] = pd.to_datetime(df["datetime"]).dt.floor("h")

    df = df.drop_duplicates(subset=["datetime"]).sort_values("datetime")

    return df

def fetch_entsoe_fr_range(start: datetime, end: datetime) -> pd.DataFrame:

    """

    Pulls all hours in [start, end] inclusive, by calling month windows with exclusive month ends.

    Then filters to the exact requested window.

    """

    parts = []

    # Use month ends at 00:00 of next month to avoid losing last day

    for m_start, m_end_excl in month_windows(start, end):

        # ENTSO-E wants exclusive end; good practice is 00:00 next day/month

        dfm = fetch_entsoe_fr_period(m_start, m_end_excl)

        parts.append(dfm)

    if not parts:

        return pd.DataFrame(columns=["datetime","FR_price"])

    df = (pd.concat(parts, ignore_index=True)

            .drop_duplicates(subset=["datetime"])

            .sort_values("datetime"))

    # Filter to the exact range requested, inclusive
    df = df[(df["datetime"] >= start) & (df["datetime"] <= end)].reset_index(drop=True)

    return df
 

In [82]:
df_fr = fetch_entsoe_fr_range(START_DATE, END_DATE)

print(df_fr.head())
print(df_fr.tail())
print(df_fr.dtypes)

             datetime  FR_price
0 2025-01-01 00:00:00     18.92
1 2025-01-01 01:00:00     16.66
2 2025-01-01 02:00:00     13.10
3 2025-01-01 03:00:00      5.90
4 2025-01-01 04:00:00      9.27
                datetime  FR_price
4988 2025-08-01 19:00:00    103.02
4989 2025-08-01 20:00:00    102.52
4990 2025-08-01 21:00:00     95.41
4991 2025-08-01 22:00:00     97.80
4992 2025-08-01 23:00:00     77.08
datetime    datetime64[ns]
FR_price           float64
dtype: object


In [83]:
#  Load UK hourly from Excel -
df_uk = pd.read_excel(os.path.join(raw_data_dir, "UK_day_ahead_prices.xlsx"))
df_uk = df_uk.rename(columns={df_uk.columns[1]: "UK_price"})
df_uk["datetime"] = pd.to_datetime(df_uk["datetime"]).dt.floor("h")

In [84]:
# check for unique datetime records
print("FR hours:", df_fr["datetime"].nunique())
print("UK hours:", df_uk["datetime"].nunique())

FR hours: 4993
UK hours: 5088


In [85]:
# Merge on datetime (inner join = only matching hours kept)
merged_df = pd.merge(
  df_fr[["datetime", "FR_price"]],
  df_uk[["datetime", "UK_price"]],
  on="datetime",
  how="inner"
).sort_values("datetime").reset_index(drop=True)

# Cleaning Fix
# Drop invalid or zero/negative prices
merged_df = merged_df[(merged_df["UK_price"] > 0) & (merged_df["FR_price"] > 0)]

# Drop duplicates just in case
merged_df = merged_df.drop_duplicates(subset=["datetime"])

# Compute log returns for sanity filtering
rets = np.log(merged_df[["UK_price", "FR_price"]]).diff()

# keep rows where returns are within ±1 (≈±170% per hour), otherwise drop as bad data
mask = (rets["UK_price"].abs() < 1) & (rets["FR_price"].abs() < 1)
merged_df = merged_df.loc[mask].copy()

# Sort again and reset index
merged_df = merged_df.sort_values("datetime").reset_index(drop=True)

In [86]:
#Inspect results
print(merged_df.head())
print(merged_df.tail())
print(merged_df.shape)

             datetime  FR_price  UK_price
0 2025-01-01 01:00:00     16.66    70.001
1 2025-01-01 02:00:00     13.10    74.074
2 2025-01-01 03:00:00      5.90    78.937
3 2025-01-01 04:00:00      9.27    62.961
4 2025-01-01 05:00:00     10.04    61.395
                datetime  FR_price  UK_price
4224 2025-07-31 19:00:00    114.45    69.859
4225 2025-07-31 20:00:00    110.27    86.063
4226 2025-07-31 21:00:00     98.32    96.437
4227 2025-07-31 22:00:00    102.22    76.596
4228 2025-07-31 23:00:00     93.47    79.206
(4229, 3)


In [87]:
#Check for missing hours - Test
import pandas as pd

# Use merged dataset
df = merged_df.copy()

# Create expected hourly timeline between min and max datetime
expected_hours = pd.date_range(df["datetime"].min(), df["datetime"].max(), freq="h")

# Identify missing hours
missing_hours = expected_hours.difference(df["datetime"])
print(f"Total missing hours: {len(missing_hours)}")
print("\nFirst 20 missing hours:")
print(missing_hours[:20])

# Group missing hours by day to see clustering
missing_df = pd.DataFrame(missing_hours, columns=["datetime"])
missing_df["date"] = missing_df["datetime"].dt.date
missing_count_per_day = missing_df.groupby("date").size().sort_values(ascending=False)
print("\nMissing hours per day (top 20 days with most missing):")
print(missing_count_per_day.head(20))

Total missing hours: 858

First 20 missing hours:
DatetimeIndex(['2025-01-06 03:00:00', '2025-01-06 06:00:00',
               '2025-01-07 00:00:00', '2025-01-07 03:00:00',
               '2025-01-07 04:00:00', '2025-01-07 05:00:00',
               '2025-01-14 03:00:00', '2025-01-20 08:00:00',
               '2025-01-24 03:00:00', '2025-01-26 10:00:00',
               '2025-01-26 12:00:00', '2025-01-27 02:00:00',
               '2025-01-27 03:00:00', '2025-01-27 04:00:00',
               '2025-01-27 05:00:00', '2025-01-28 03:00:00',
               '2025-01-28 04:00:00', '2025-01-28 05:00:00',
               '2025-01-29 02:00:00', '2025-01-29 03:00:00'],
              dtype='datetime64[ns]', freq=None)

Missing hours per day (top 20 days with most missing):
date
2025-06-05    18
2025-06-08    15
2025-06-03    14
2025-05-25    14
2025-06-06    13
2025-06-12    13
2025-03-30    13
2025-05-10    12
2025-05-06    12
2025-06-01    11
2025-05-22    11
2025-04-06    11
2025-07-20    11
2025-05-

In [88]:
import os
output_path = os.path.join(
    processed_data_dir,
    "UK_FR_day_ahead_hourly_010125-31072025.csv"
)
merged_df.to_csv(output_path, index=False)
print(f"Merged hourly dataset saved to {output_path}")

Merged hourly dataset saved to /Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main/data/processed/UK_FR_day_ahead_hourly_010125-31072025.csv
